In [ ]:
%%bash
echo "---
version: '2'

services:
  zookeeper:
    image: confluentinc/cp-zookeeper:7.1.0
    hostname: zookeeper
    container_name: zookeeper
    ports:
      - \"2181:2181\"
    environment:
      ZOOKEEPER_CLIENT_PORT: 2181
      ZOOKEEPER_TICK_TIME: 2000

  broker:
    image: confluentinc/cp-kafka:7.1.0
    hostname: broker
    container_name: broker
    depends_on:
      - zookeeper
    ports:
      - \"29092:29092\"
    environment:
      KAFKA_BROKER_ID: 1
      KAFKA_ZOOKEEPER_CONNECT: 'zookeeper:2181'
      KAFKA_LISTENER_SECURITY_PROTOCOL_MAP: PLAINTEXT:PLAINTEXT,PLAINTEXT_HOST:PLAINTEXT
      KAFKA_ADVERTISED_LISTENERS: PLAINTEXT://broker:9092,PLAINTEXT_HOST://localhost:29092
      KAFKA_OFFSETS_TOPIC_REPLICATION_FACTOR: 1
      KAFKA_GROUP_INITIAL_REBALANCE_DELAY_MS: 0

  schema-registry:
    image: confluentinc/cp-schema-registry:7.1.0
    hostname: schema-registry
    container_name: schema-registry
    depends_on:
      - broker
    ports:
      - \"8081:8081\"
    environment:
      SCHEMA_REGISTRY_HOST_NAME: schema-registry
      SCHEMA_REGISTRY_KAFKASTORE_BOOTSTRAP_SERVERS: 'broker:9092'
      SCHEMA_REGISTRY_LOG4J_ROOT_LOGLEVEL: WARN" > docker-compose.yml

In [ ]:
%%bash
docker-compose up -d

In [ ]:
%pip install confluent-kafka

In [ ]:
from confluent_kafka import Producer

In [ ]:
config = {
          "bootstrap.servers":"127.0.0.1:29092",
          "auto.offset.reset":"earliest",
          "group.id":"kafka-tutorial"
         }

In [ ]:
producer = Producer(config)

In [ ]:
producer.produce("tutorial", key="a", value="Hello world!")
producer.produce("tutorial", key="a", value="Hello again!")
producer.produce("tutorial", key="a", value="Goodbye!")


In [ ]:
from confluent_kafka import Consumer

In [ ]:
consumer = Consumer(config)
consumer.subscribe(["tutorial"])
for i in range(5):
    evt = consumer.poll(1.0)
    if evt is None:
        pass
    else:
        print(evt.value())